In [1]:
KotlinVersion.CURRENT             // 코틀린 현재 버전 

1.8.0

In [2]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")

In [3]:
:classpath

Current classpath (16 paths):
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/lib-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/api-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.8.0-dev-3517.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlinx-serialization-json-jvm-1.3.3.jar
/Users/a06

In [4]:
import kotlinx.coroutines.*

# 1. 코루틴 알아보기

## 1-1 코루틴 기본 알아보기

- 코루틴스코프를 별도로 지정해야 코루틴을 실행할 수 있다. 
- 그래서 기존 함수에서 코루틴스코프를 먼저 지정해야 한다. 

### 프로세스 

- 실행중인 애플리케이션의 인스턴스이다. 

### 스레드 

- 실행 스레드는 프로세스가 실행할 일련의 명령을 포함하면 최소한 하나의 스레드를 포함한다. 
- 코틀린의 메인 함수도 코루틴으로 처리한다. 

### 코루틴(coroutine) 
- 경량스레드이고 스레드와 비슷한 라이프 사이클을 갖는다.
- 코루틴은 스레드 안에서 실행한다

### 동시성
- 넌블럭킹 : 중단가능한 연산을 실행하면 블러킹하지 않고 실행을 잠시 중단하는 것이다. 그리고 재실행할 때 다시 스레드를 사용해서 처리한다. 

## 1-1-1 코루틴을 처리하도록 코루틴을 생성

- runBlocking 블록에서도 현재 쓰레드는 그대로 진행된다. 
- runBlocking 블록 내 어떠한 코루틴 실행이 병렬이나 비동기로 실행되더라도 해당 블록들이 완전히 종료되어야 runBlocking 블록이 종료된다는 것이다

### 코루틴 스코프를 만들면 코루틴컨텍스트가 만들어진다,

- 코루틴디스패치 : 코루틴을 시작하거나 재개할 스레드를 결정한다. 

In [5]:
fun main() = runBlocking { 
    println("코루틴 처리 : " + coroutineContext)   
}

In [6]:
main()

코루틴 처리 : [BlockingCoroutine{Active}@10b04899, BlockingEventLoop@6142aef0]


## 1-1-2 코루틴 스코프 내에 코루틴빌더로 코루틴을 추가 생성 

- 코루틴스코프 내에 코루틴을 만들어서 동시에 작업이 가능하다

In [7]:
fun main() = runBlocking {           // 코루틴 스코프
    launch {                         // 코루틴 빌더로 새로운 코루틴 생성 
        delay(1000L)                 // 일시중단함수 지연 
        println("World!")            // 출력하기 
    }
    println("Hello")                 // main 코루틴 출력 
}

In [8]:
main()

Hello
World!


## 1-1-3  코루틴 스코프 내에 코루틴빌더로 일시중시함수를 코루틴으로 처리 가능

- 코루틴 스코프 내에 코루틴을 생성할 때 일시중단함수를 사용해서 처리가 가능하다. 
- 이때 launch 코루틴 빌더로 람다임시중단함수로 전달한다. 
- 람다일시중단함수에 일시중단함수 실행을 내포해서 처리한다.

In [9]:
fun main() = runBlocking {            // this: CoroutineScope
    launch { doWorld() }
    println("Hello")
}

                                      // this is your first suspending function
suspend fun doWorld() {
    delay(1000L)
    println("World!")
}

In [10]:
main()

Hello
World!


## 1-2  일시중단함수 

- 함수 앞에 suspend 키워드를 사용해서 코루틴에서 처리하는 일시 중단함수를 작성할 수 있다. 
- 보통 일시중단함수는 호출한 순서대로 순차적으로 실행한다.  일시중단할 때 스레드를 블러킹하지 않는다 

## 1-2-1  일시중단함수 정의 

- 일시중단함수 내에 일시중단함수를 사용할 수 있음

In [11]:
suspend fun getName() : String {
    delay(100)
    return "코루틴"
}

suspend fun getLastName() : String {
    delay(100)
    return "코틀린"
}

### 코루틴이 실행되는 영역을 위해 runBlocking으로 영역을 설정

- 코루틴 영역이 설정해야 내부에서 일시중단함수 등이 실행된다. 

In [12]:
fun main() = runBlocking {
    val name = getName()
    val lastName = getLastName()
    println("Hello, $name $lastName")
}

In [13]:
main()

Hello, 코루틴 코틀린


## 1-2-2 코루틴 빌더  사용하기 

- 코루틴 빌더는 일시중단람다를 받아 실행하는 코루틴을 생성하는 함수 
- async : 결과가 예상되는 코루틴을 시작 결과는 Deferred<T>로 반환
- launch : 결과를 반환하지 않는 코루틴 결과로 Job 반환, 이 Job으로 실행을 취소하기 위해 사용
- runBlocking : 블러킹 코드를 일시 중지 가능한 코드로 연결, 보통 메인과 유닛테스트에서 사용 

###  동시에 일시중단함수를 호출하기

In [14]:
fun main() = runBlocking {
    val name = launch {getName() }
    val lastName = launch {getLastName()}
    println("Hello, $name $lastName")
}

In [15]:
main()

Hello, StandaloneCoroutine{Active}@48c6718f StandaloneCoroutine{Active}@13a0522b


###  동시에 실행하고 반환값 처리 
- 반환값은 await 메서드를 사용함

In [16]:
fun main() = runBlocking {
    val name = async {getName() }
    val lastName = async {getLastName()}
    println("Hello, ${name.await()} ${lastName.await()}")
}

In [17]:
main()

Hello, 코루틴 코틀린


## 1-2-3 코루틴 빌더로 코루틴 실행 함수 정의 

- launch 코루틴빌더로 처리하면 job에 결과를 반환한다. 
- 코루틴 처리된 결과가 내포된다. 
- 이 코루틴의 결과를 가지고 join, cancel 등의 메소드로 코루틴 결과를 처리할 수 있다

In [18]:
import kotlinx.coroutines.*                               // 코루틴 패키지

suspend fun createCoroutines(amount:Int)= runBlocking { 
    var jobs = ArrayList<Job>()                           // 배열의 작업 결과를 저장
    for (i in 1..amount) {
        jobs += launch {                                  // 코루틴 처리 후 결과 저장 
            print("...")
            delay(1000)
        }
    }
    
    jobs.forEach {
        it.join()                                         // 코루틴 실행할 때까지 기다린다. 
    }
    println()
    println(jobs)                                         // 코루틴 처리 결과 
}


In [19]:
fun main() = runBlocking {
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
    print("start")
    createCoroutines(10)                                  // 코루틴 일시중단함수 실행 
    println("end")
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
}

In [20]:
main()

코루틴 처리 : [BlockingCoroutine{Active}@3dc2446c, BlockingEventLoop@e5c8ade]
start..............................
[StandaloneCoroutine{Completed}@615c4f82, StandaloneCoroutine{Completed}@1003fd9a, StandaloneCoroutine{Completed}@3fa9c1db, StandaloneCoroutine{Completed}@48c157d2, StandaloneCoroutine{Completed}@4b622117, StandaloneCoroutine{Completed}@4402facc, StandaloneCoroutine{Completed}@35d44c8e, StandaloneCoroutine{Completed}@77e9f221, StandaloneCoroutine{Completed}@2d590f52, StandaloneCoroutine{Completed}@7ede54af]
end
코루틴 처리 : [BlockingCoroutine{Active}@3dc2446c, BlockingEventLoop@e5c8ade]


## 1-2-4  일반함수와 일시중단함수를 코루틴에서 처리

- launch 함수로 반환값 없이 실행하는 코루틴을 처리하다
- 코루틴 내부에서 일반함수와 일시중단함수를 호출해서 처리할 수 있다. 

In [21]:
fun task1() = println("일반함수 task 1 : " 
                     + Thread.currentThread().name)       // 첫번째 함수 

fun task2() = println("일반함수 task 2 : "                  // 두번째 함수 
                     + Thread.currentThread().name)

In [22]:
suspend fun task3() =  println("일수중단함수 task 3 : "      // 일시중단함수 정의
                     + Thread.currentThread().name)

val task4 = suspend {println("일시중단함수 task 4 : "        // 람다표현식도 일시중단함수 정의 가능 
                     + Thread.currentThread().name)}

In [23]:
val rb = runBlocking { 
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
    launch {                                              // 코루틴 빌더로 함수처리
        delay(300)
        task1()                                           // 일반함수 호출 
    }
    launch {                                              // 코루틴 빌더로 함수처리
        task2()                                           // 일반함수 호출 
        delay(300)
    }
    launch {                                              // 코루틴 빌더로 함수처리
        delay(300)
        task3()                                           // 일시중단함수 
    }
    launch {                                              // 코루틴 빌더로 함수처리
        task4()                                           // 일시중단함수
        delay(300)
    }
}                                                         // 메인 스레드 사용

코루틴 처리 : [BlockingCoroutine{Active}@601ee567, BlockingEventLoop@3cc2776e]
일반함수 task 2 : Thread-89
일시중단함수 task 4 : Thread-89
일반함수 task 1 : Thread-89
일수중단함수 task 3 : Thread-89


## 1-3 디스패처 알아보기¶

- Dispatcher는 CoroutineContext를 상속받아 어떤 쓰레드를 이용해서 동작할 것인지 정의해둔 것 입니다.

- None : 실행된 코루틴 스코프에게 상속받는다.
- Unconfined : 호출 쓰레드에서 실행하지만 멈췄다가 재개할 경우 재개한 쓰레드에서 실행한다.
             -> 특정 쓰레드에 국한되지 않는 경우 적합

- Main : 메인 쓰레드에서 실행한다.
         -> 안드로이드에서 UI를 바꿀 때 적합, Job이 무거우면 부적합

- IO : 네트워크, 디스크 사용 할때 사용합니다. 파일 읽고, 쓰고, 소켓을 읽고, 쓰고 작업을 멈추는것에 최적화되어 있습니다.
- Default : Thread Pool에 할당한다. (GlobalScope.launch { }도 마찬가지)

## 1-3-1 디스패처 상태 확인하기 

In [24]:
fun main() { 
    runBlocking {
        launch {
            println(" 미지정 일때  : " + coroutineContext)             // 코루틴 환경구성 
            println(" launch(미지정) : ${Thread.currentThread().name}")
        }

        launch(Dispatchers.Unconfined) {
            println("Unconfined : " + coroutineContext)             // 코루틴 환경구성 
            println("launch(Unconfined) : ${Thread.currentThread().name}")
        }

       //launch(Dispatchers.Main) {   //안드로이드 용
       //    println("launch(Dispatchers.Main) : ${Thread.currentThread().name}")
       //}

        launch(Dispatchers.IO) {
            println(" IO : " + coroutineContext)             // 코루틴 환경구성 
            println("launch(IO) : ${Thread.currentThread().name}")
        }

        launch(Dispatchers.Default) {
            println("Default : " + coroutineContext)             // 코루틴 환경구성 
            println("launch(Default) : ${Thread.currentThread().name}")
        }
    }
}

main()

Unconfined : [StandaloneCoroutine{Active}@49571da8, Dispatchers.Unconfined]
launch(Unconfined) : Thread-93
 IO : [StandaloneCoroutine{Active}@27f8dd3c, Dispatchers.IO]
 미지정 일때  : [StandaloneCoroutine{Active}@63004ba2, BlockingEventLoop@781df849]
launch(IO) : DefaultDispatcher-worker-1
Default : [StandaloneCoroutine{Active}@64a44eee, Dispatchers.Default]
 launch(미지정) : Thread-93
launch(Default) : DefaultDispatcher-worker-3


### 코루틴디스패처와 스레드 풀 정의 

-  newSingleThreadContext : 단일 스레드로 디스패처 생성
-  newFixedThreadPoolContext : 지정된 크기의 스레드 풀로 디스패처 생성 

In [25]:
fun main() { 
    val single = newSingleThreadContext("aaa" )
    runBlocking {
        launch(single) {
            delay(100)
            println("launch(Default) : ${Thread.currentThread().name}")
        }
    }
}

In [26]:
main()

launch(Default) : aaa


In [27]:
fun main() { 
    val multiple = newFixedThreadPoolContext(4, "bbb")
    runBlocking {
        launch(multiple) {
            delay(100)
            println("launch(Default) : ${Thread.currentThread().name}")
        }
    }
}

In [28]:
main()

launch(Default) : bbb-1


## 1-3-2 코루틴 컨텍스트 조회하기 

In [29]:
fun main() = runBlocking {                                     // 런블러킹 컨텍스트 확인 
    println("runBlocking : ${Thread.currentThread().name}")    
    println("컨텍스트 : " + coroutineContext)                                      
    println("상태 : " +isActive)                                // 잡상태 확인
    
    val g = GlobalScope.launch {                               // 전역스코프 컨텍스트 확인 
        delay(300)
        println("전역스코프 : ${Thread.currentThread().name}")
        println("컨텍스트 : " +coroutineContext)
        println("상태 : " +isActive)                            // 잡상태 확인 
    }
    println("전역스코프 반환 : " + g)
    val c =CoroutineScope(Dispatchers.Default).launch {        // 코루틴스코프 컨텍스트 확인
        delay(500)
        println("코루틴 스코프 : ${Thread.currentThread().name}")
        println("컨텍스트 : " +coroutineContext)
        println("상태 : " +isActive)                             // 잡상태 확인    
    }
    println("코르틴 스코프 반환 : " + c)
    delay(1000)
}

main()

runBlocking : Thread-116
컨텍스트 : [BlockingCoroutine{Active}@76ebf70b, BlockingEventLoop@773e97cf]
상태 : true
전역스코프 반환 : StandaloneCoroutine{Active}@4dc5de92
코르틴 스코프 반환 : StandaloneCoroutine{Active}@eeb0150
전역스코프 : DefaultDispatcher-worker-3
컨텍스트 : [StandaloneCoroutine{Active}@4dc5de92, Dispatchers.Default]
상태 : true
코루틴 스코프 : DefaultDispatcher-worker-3
컨텍스트 : [StandaloneCoroutine{Active}@eeb0150, Dispatchers.Default]
상태 : true


## 1-4   스레드와 코루틴 실행 비교하기 

In [30]:
fun main() {
    Thread(Runnable {                                   // 3개의 러너블 객체를 만들어서 스레드 처리               
        for (i in 0..2) {
            println("쓰레드 처리  : $i "                   // 3개의 쓰레드를 처리 
                    + Thread.currentThread().name)
            Thread.sleep(500L)                          // 스레드 중단 : 현재 스레드 일시중단 
        }
    }).start()                                          // start 메소드로 스레드 시작

    GlobalScope.launch {                                // 코루틴 전역스코프로 처리 
        repeat(3) {                                     // 3개의 일지중단함수 만들고 코루틴 생성 
            println("코루틴처리   : $it " 
                    + Thread.currentThread().name)      // 코루틴 내의 dispatchre로 처리 
            delay(500L)                                 // 일시 중단 다른 작업 수행 
        }
    }
    
    Thread.sleep(3000L)                                 // 어플리케이션 중단 
}

main()

쓰레드 처리  : 0 Thread-121
코루틴처리   : 0 DefaultDispatcher-worker-3
쓰레드 처리  : 1 Thread-121
코루틴처리   : 1 DefaultDispatcher-worker-3
쓰레드 처리  : 2 Thread-121
코루틴처리   : 2 DefaultDispatcher-worker-3
